 ## Import packages and modules

In [12]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col
import eeDatabase_collectionInfo as eedb_colinfo

# ee.Authenticate()
ee.Initialize(project = "dri-apps")

Map = geemap.Map()

## Define Parameters and Load datasets

In [13]:
# ------------------------------------- Define parameters -----------------------------------------------
# Define time period to export
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2020, 2, 1)

# -------------------------------- Define input Image Collection ----------------------------------------
# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'GridMET'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'precip'

# ------------------------------- Define input Feature Collection ---------------------------------------
# Define input path for Feature Collection
in_fc_path = 'projects/climate-engine-pro/assets/ce-fields-polygons/KSFields240105'
in_fc = ee.FeatureCollection(in_fc_path)

# ------------------------------ Define mask, if applicable --------------------------------------------
# For BLM, we will apply mask to field offices, district offices, and state offices, but not to allotments
# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'

## Define additional parameters required for functions and output properties below

In [14]:
# Define land unit names
if(in_fc_path == 'projects/climate-engine-pro/assets/ce-fields-polygons/KSFields240105'):
    land_unit_long = 'KS_Fields_Polygons'
    land_unit_short = 'KS_Fields'
    tile_scale = 1
    in_fc_id = 'ID'
    fc_mask = False
elif(in_fc_path == 'projects/climate-engine-pro/assets/ce-fields-polygons/NEFields240105'):
    land_unit_long = 'NE_Fields_Polygons'
    land_unit_short = 'NE_Fields'
    tile_scale = 1
    in_fc_id = 'ID'
    fc_mask = False

# Pull out additional variables needed to run exports
in_ic_paths = eedb_colinfo.in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = eedb_colinfo.in_ic_dict.get(in_ic_name).get('var_type')
var_units = eedb_colinfo.var_dict.get(var_name).get('units')
out_path = f"projects/climate-engine-pro/assets/field-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

# Apply mask for remote sensing datasets on larger boundaries
if fc_mask == True & eedb_colinfo.in_ic_dict.get(in_ic_name).get('ic_mask') == True:
    mask = True
else:
    mask = False

## Create database image collection and append time-series images

In [15]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    eedb_cor.initialize_collection(out_path = out_path, properties = properties)

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:

    print(f"Appending to Image Collection for dates {start_date} - {end_date}")

    # Get dates for image collection based on start and end date
    dates = eedb_cor.get_collection_dates(in_ic_paths = in_ic_paths, start_date = start_date, end_date = end_date)

    # Loop over selected dates to pre-process and export
    for date in dates:
        
        print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
        
        # Parse date for ID
        date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')
        
        # Create dictionary of properties for image    
        properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale}
        if mask == True:
            properties['mask_path'] = mask_path
        elif mask == False:
            properties['mask_path'] = 'None'
        
        eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

{
  "id": "projects/climate-engine-pro/assets/field-database/ksfields-gridmet-precip",
  "name": "projects/climate-engine-pro/assets/field-database/ksfields-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2024-01-06T21:32:05.220354Z"
}
{
  "id": "projects/climate-engine-pro/assets/field-database/ksfields-gridmet-precip",
  "name": "projects/climate-engine-pro/assets/field-database/ksfields-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2024-01-06T21:32:05.220354Z"
}
Appending to Image Collection for dates 2020-01-01 00:00:00 - 2020-02-01 00:00:00
Running  2020-01-05 01:00:00
Running  2020-01-10 01:00:00
Running  2020-01-15 01:00:00
Running  2020-01-20 01:00:00
Running  2020-01-25 01:00:00
Running  2020-01-30 01:00:00


## Check completeness of image collections and re-run any images that are missing

In [32]:
# Get list of all dates
all_dates = eedb_cor.get_collection_dates(in_ic_paths, start_date, end_date)

# Get list of dates from collection
coll_dates = ee.ImageCollection(out_path).aggregate_array('system:time_start').distinct().getInfo()

# Get list of dates missing from collection
miss_dates = sorted(set(all_dates) - set(coll_dates))
print("These dates are missing and will be rerun ", miss_dates)

for date in miss_dates:
    print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
    
    # Parse date for ID
    date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')

    # Create dictionary of properties for image    
    properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale, 'mask': mask}
    if mask == True:
        properties['mask_path'] = mask_path
    elif mask == False:
        properties['mask_path'] = 'None'
        
    eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

EEException: ImageCollection.load: ImageCollection asset 'projects/climate-engine-pro/assets/blm-database/blmallotments-gridmet-windspeed' not found (does not exist or caller does not have access).